In [72]:
import itertools, nltk, numpy as np, pandas as pd
from collections import OrderedDict, Counter
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree

# constants and parameters

amino_acids = "ACDEFGHIKLMNPQRSTVWY"
amino_acids_sorted = ''.join(sorted(tuple(amino_acids))) # alphabet sort
aa_bigrams = tuple(itertools.product(amino_acids, repeat=2))
spec_subseqs = ("DEKHR", "ILV", "FHWY", "DERKQN", "AGHPSTY", "CFILMVW", "KRH", "DE", "ACDGST", "EHILKMNPQV", "FRWY", "ILVA", "AGVILFP", "YMTS", "HNQW")
df = pd.read_csv("train.csv")
all_inputs, all_targets = df["input"].to_numpy(), df["target"].to_numpy()
inputs, test_inputs, targets, test_targets = train_test_split(all_inputs, all_targets, test_size=0.33, shuffle=True)
#df.head()
print(len(df))

def feat_aac(seq : str) -> np.array:
    n = len(seq)
    d = OrderedDict.fromkeys(amino_acids, value=0)
    it = OrderedDict(Counter(seq))
    for k, v in it.items():
        d[k] = v / n
    return np.array(list(d.values()))

def feat_dpc(seq : str) -> np.array:
    d = OrderedDict.fromkeys(aa_bigrams, value=0)
    it = OrderedDict(Counter(nltk.bigrams(seq)))
    for k, v in it.items():
        d[k] = v
    n = sum(d.values())
    for k in d.keys():
        d[k] /= n
    return np.array(list(d.values()))

def feat_qso(seq : str, maxlag : int = 5, weight : float = 0.1) -> np.array:
    aac, n = feat_aac(seq), len(amino_acids)
    sum_aac = sum(aac)
    d = np.zeros((n,n))
    for i, j in itertools.product(range(n), repeat=2):
        d[i,j] = (aac[j] - aac[i])**2
    r = np.zeros((maxlag,))
    for i in range(maxlag):
        r[i] = 0
        for j in range(n - maxlag):
            r[i] += d[j,i+j]**2 
    sum_r = sum(r)
    temp = sum_aac + weight*sum_r
    qso = aac / temp
    return qso

features = {"aac" : feat_aac, "dpc" : feat_dpc, "qso" : feat_qso }
sub_models = OrderedDict()



1250


In [73]:
len(inputs)

837

In [74]:
# initial models
for f in features.keys(): # make independent model by each feature
    sub_models[f] =  make_pipeline(StandardScaler(), SVC(gamma = "auto", probability=True, random_state=0))
final_model = RandomForestClassifier(n_estimators = 1000, random_state=0)

transformed_X = OrderedDict() # extract features from aa-sequence
for k,v in features.items():
    transformed_X[k] = np.array(list(map(lambda item: v(item), inputs)))
    print("%s\t%d" % (k, len(transformed_X[k])))
    sub_models[k].fit(transformed_X[k], targets)

predict_func = lambda item: sub_models[item[0]].predict_proba(item[1])[:,1]
predicted_X = list(map(predict_func, transformed_X.items()))
predicted_X = OrderedDict([(k, v) for k, v in zip(features.keys(), predicted_X)])

X_final = np.array(list(predicted_X.values())).T
final_model.fit(X_final, targets) # got learned model

aac	837
dpc	837
qso	837


RandomForestClassifier(n_estimators=1000, random_state=0)

In [61]:
test_seq_true = "MSWDKRVAVNYAKTHAGSHSQGRCAEFTRKAIQAGGITLGHTYHAKDYGPMLRSAGFTAIGTYEMPREGDVIIIQPYAGGNPSGHMAIYDGAEWYSDFKQRDMWAGPGYRAARPSYTIYRKN"
test_seq_false = "MTVVCFNPSYTSNKNSGRMLHARVAQRNPRITIMSEQIKNVSDASFEADVLKSGQPVLVDYWAAWCGPCKMIAPILEEVAAEYAGRLTVAKLNVDENQDTAAKYGVRGIPTLMLFKDGQAAATKVGALSKSQLTAFLDGAL"
seqs = [test_seq_true, test_seq_false]
def eval_features(seq):
    transform_func = lambda f: f(seq)
    predict_func = lambda item: sub_models[item[0]].predict_proba([item[1]])[0][1]
    temp_values = list(map(transform_func, features.values()))
    temp =  OrderedDict(
        [(k, v) for k, v in zip(features.keys(), temp_values)])
    result = list(map(predict_func, temp.items()))
    return np.array(result)


feats = list(map(eval_features, seqs))
final_model.predict(list(map(eval_features, seqs)))

array([ True, False])

In [77]:
# here we compare results with Bastion6 predictions (not a verified proteins!)
compare_df = pd.read_csv("compare.csv")
#compare_df.head()
cmp_seqs = compare_df["input"].to_numpy()
prediction = final_model.predict(list(map(eval_features, cmp_seqs)))
compare_df['prediction'] = prediction

In [78]:
compare_df.head(20)

,Unnamed: 0,id,input,AAC,DPC,QSO,target,prediction
0,0,F3B38_RS06900,MAFEVIAALALPASCRVDQRVPKRMLIENGAPTAADKRLLTDTIEE...,0.041,0.167,0.015,False,False
1,1,F3B38_RS06905,MKMIDATSAEAQSADLTAANIMQIKALFPELITEGANGVAVNVDVL...,0.677,0.594,0.772,True,False
2,2,F3B38_RS06910,MKLHFEPNLDYQLQAIEAICELFRGQESCRTEFTVTMKLPDQQLTL...,0.058,0.135,0.290,False,False
3,3,F3B38_RS06920,MINDVHEPLEQYSFHFKNAHASNTSDFFEDLVRRSGVDENANITTV...,0.764,0.792,0.653,True,False
4,4,F3B38_RS06925,MANDLDEVTGPINEQGRDINVISKQISVQVGTGSLIFEIALWAVLP...,0.025,0.035,0.032,False,False
5,5,F3B38_RS06930,MSQYQDDIKTVAGLKDQQGSAWNAINPESAARMRAQNKFKTGLDIA...,0.611,0.552,0.595,True,False
6,6,F3B38_RS06935,MPTSDTAASNAATRPLTQQDYKTLSLAALGGALEFYDFIIFVFFAN...,0.024,0.026,0.009,False,False
7,7,F3B38_RS06940,MHIMIVGASRGLGRALVDGMLADGHAVIGVSRQQPADLPAGQGTQL...,0.053,0.091,0.019,False,False
8,8,F3B38_RS06945,MLLSLDDPLWPTLEGGYRVPYDVAEALKAMQAGEDVWHELWEELHH...,0.262,0.261,0.123,False,False
9,9,F3B38_RS06950,MRASHNHQGAIDGERYAGIMPGHAGGAGYRLFLLPGEADALPWQAA...,0.830,0.950,0.756,True,True


In [31]:
X_evaluate = list(map(eval_features, inputs))
len(X_evaluate)

1250

In [33]:
from sklearn import ensemble, model_selection, metrics, datasets,tree
model_selection.cross_val_score(final_model, X_evaluate, targets,cv=10,scoring='accuracy').mean()

0.9960000000000001

In [75]:
test_predictions = final_model.predict(list(map(eval_features, test_inputs)))

In [76]:
pairs = list(zip(test_targets, test_predictions))
count = len(pairs)
print(count)
tn =0; tp = 0; fn=0; fp = 0;
for t, p in pairs:
    if t:
        if p:
            tp += 1
        else: 
            fn += 1
    else:
        if p:
            fp += 1
        else:
            tn += 1
print ("TP: %.1f, TN: %.1f, FP: %.1f, FN: %.1f" % (tp/count*100, tn/count*100, fp/count*100, fn/count*100))
print ("TP: %d, TN: %d, FP: %d, FN: %d" % (tp, tn, fp, fn))

413
TP: 6.8, TN: 88.1, FP: 1.7, FN: 3.4
TP: 28, TN: 364, FP: 7, FN: 14
